In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt

pd.set_option('display.float_format', '{:.4f}'.format)
%precision 4
%config InlineBackend.figure_format = 'retina'

import pandas_datareader as pdr
import requests_cache
session = requests_cache.CachedSession(expire_after='1D')

ModuleNotFoundError: No module named 'pandas_datareader'

In [3]:
import sys
!conda install --yes --prefix {sys.prefix} pandas_datareader


PackagesNotFoundError: The following packages are not available from current channels:



Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



  - pandas_datareader

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [ ]:
ff = (
    pdr.get_data_famafrench(
        symbols='F-F_Research_Data_Factors', # get monthly FF factors
        start='1900', # return all available data
        session=session # use my cache file to reduce downloads
    )[0] # slice the factor data frame
    .div(100) # convert it to decimals
    .assign(Mkt = lambda x: x['Mkt-RF'] + x['RF']) # calculate market return
    [['Mkt', 'RF']] # slice only market return and risk-free rate
)

# Consider only 1999 through today

In [ ]:
# manipulation of data to only consider Fama-French data from 1999 to today
df_starting_99 = ff['200001':'202209'] # TODO: check with professor during office hours if this is the intended splice of timeframe

### LS Investment Strategy

In [ ]:
# function used in reading the LS Strategy plot and retrieving the portfolio final value
# citation: https://stackoverflow.com/questions/20130768/retrieve-xy-data-from-matplotlib-figure
def get_final_value(plot=plt):
    ax = plt.gca()
    line = ax.lines[0]
    y_data = line.get_ydata()
    final_value = round(y_data[-1], 2)
    return final_value

In [ ]:
df_starting_99['Mkt'].add(1).cumprod().mul(18000).plot() # invests $18,000 into the market all at the start
plt.ylabel('Value ($)')
plt.title('LS Investment Strategy:\n$18,000 Lump Sum Investment in the Mkt [2000-2022]', fontsize=10, fontweight='bold')
plt.figtext(0.335, 0.3, 'Note: Decrease in value from 2000 to 2003', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
fin_val = get_final_value()
plt.figtext(1.03, 0.5, f'Final Value = ${fin_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

### DCA Investment Strategy

In [ ]:
# input: fama-french market data with properly spliced timeframe, starting investment
# assumes that $1000 investments max will be made into the market, all money in the savings account earns risk free interest
# output: pandas series of the overall value of the portfolio, the final value of the portfolio
def dca_1000(data, starting_investment):
    investment_remaining = starting_investment # the amount that is being invested into the DCA strategy
    for date, (mkt, rf) in data[['Mkt', 'RF']].iterrows(): # loop down the data frame of market returns
        if date == data.index[0]:
            dca_bgn = 0
            data['dca_end'] = np.nan # not strictly necessary
        if investment_remaining >= 1000:
            dca_end = (dca_bgn + 1_000) * (1 + mkt)
            investment_remaining = investment_remaining - 1000 + (investment_remaining * rf) # calculates new cash amount left
        elif investment_remaining < 1000 and investment_remaining > 0: # last period of moving cash from savings to market
            dca_end = (dca_bgn + investment_remaining) * (1 + mkt)
            investment_remaining = 0
        else: # all remaining growth through the market after all investments have been made
            dca_end = dca_bgn * (1 + mkt)
        data.loc[date, 'dca_end'] = dca_end
        dca_bgn = dca_end
    final_value = round(data['dca_end'].iloc[-1], 2) # calculates the final portfolio value
    return data['dca_end'], final_value

In [ ]:
result, end_val = dca_1000(df_starting_99, 18000)

In [ ]:
#df_starting_99['dca_since_99_end'].plot()
result.plot()
plt.ylabel('Value ($)')
plt.title('DCA Investment Strategy:\n\$1,000 Monthly Using \$18,000 Total [2000-2022]', fontsize=11, fontweight='bold')
plt.figtext(0.4, 0.7, 'DCA Investment strategy factors in risk free\n interest rate earned from cash in savings account', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.figtext(0.335, 0.35, 'Note: Increase in value from 2000 to 2003', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.figtext(1.03, 0.5, f'Final Value = ${end_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

## Analysis for Question 1
Based on our calculations, the Lump Sum investment strategy's final value was \\$70,508.94 and the DCA investment strategy's final value was \\$80,049.64, a difference of \\$9,540.70 in favor of the DCA strategy. We attribute this largely to the market returns viewable between the years of 2000 to 2003. This timeframe of investments (approx 18 months for the DCA strategy) occurred while the market returns were generally negative, meaning the value of the cash invested in the LS method was depreciating whilst the cash invested gradually in the DCA method worked to the investors benefit. Scaling into the position slowly helped lower the overall average entry price. The LS method was greatly affected by the timing in which the investment was made.

## TODO: Ask professor during office hours if this analysis/discussion is sufficient ^

# Can you find a period that reverses your question 1 conclusion?

As we alluded to within our dicussion of results to question 1, the period of 2000 to 2003 saw a generally negative market return. Our prediction is that if we postponed the first investment date to the beginning of 2003, our conclusions found for question 1 will reverse (that is, the LS investment method will have a higher final value than the DCA investment method).

### LS Investment Strategy

In [ ]:
# manipulation of data to only consider Fama-French data from 2003 to today
df_starting_03 = ff['200301':'202209'] # TODO: check with professor during office hours if this is the intended splice of timeframe

In [ ]:
df_starting_03['Mkt'].add(1).cumprod().mul(18000).plot() # invests $18,000 into the market all at the start
plt.ylabel('Value ($)')
plt.title('LS Investment Strategy:\n$18,000 Lump Sum Investment in the Mkt [2003-2022]', fontsize=10, fontweight='bold')
fin_val = get_final_value()
plt.figtext(1.03, 0.5, f'Final Value = ${fin_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

### DCA Investment Strategy

In [ ]:
result, end_val = dca_1000(df_starting_03, 18000)

In [ ]:
result.plot()
plt.ylabel('Value ($)')
plt.title('DCA Investment Strategy:\n\$1,000 Monthly Using \$18,000 Total [2003-2022]', fontsize=11, fontweight='bold')
plt.figtext(1.03, 0.5, f'Final Value = ${end_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

## Analysis for Question 2
As we initially predicted, altering the start date to the beginning of 2003 reversed our conclusions from question 1. The LS method returned a final value of \\$114,237.48 while the DCA method returned a final value of \\$98,810.69. It's also interesting to note that both methods resulted in a higher final value when starting in 2003 relative to the final value when starting in 2000. As a result of moving the start date forward, the LS strategy's return grew by approximately 62\% and the DCA strategy's return grew by approximately 23\%. 

Our conclusions through both of the analyses are that if the overall market return during the DCA period is positive, the LS strategy will have a higher ending value because it maximizes the benefits of the positive market changes. In this situation, the DCA method has a higher average entry price than the LS strategy because it is buying into higher and higher prices compared to the starting price that the LS strategy entered at. If the overall market return during the DCA period is negative, the DCA strategy will have a higher ending value since the average entry price will be lower than the LS strategy.

## TODO: Ask professor during office hours if this analysis/discussion is sufficient ^

### TODO: Delete before submitting?? I'm unsure if this is a question that has to be explicitly answered or not
### ask professor during office hours
We choose between LS and DCA for retirement savings, too. For example, we can invest about `$`20,000 yearly in tax-advantaged 401(k) accounts. Your goal for this project is to convince me which is better, LS or DCA. Explain and discuss your results.

# Consider all full years from 1927 through 2021

LS immediately invests all `$`20,000 in the market. DCA immediately invests `$`20,000/12 in the market, the remainder in cash, and moves `$`20,000/12 from savings to the market each month. Each year’s last move will move the remainder of the cash account, so the next year starts with a zero balance. Which strategy has a higher final value and why? Discuss.

In [ ]:
# manipulation of data to only consider Fama-French data from 1927 through 2021
df_full = ff['192701':'202112'] 

### LS Investment Strategy

In [ ]:
# indicate which months to make ls investment
df_full['Add_Investment'] = False # default
df_full['Add_Investment'][::12] = True # adds a boolean indicator to add a lump sum investment at the start of every year

In [ ]:
def ls_per_year(data, yearly_investment=20000):
    for date, (mkt, rf, new_payment) in data[['Mkt', 'RF', 'Add_Investment']].iterrows():
        if date == data.index[0]:
            bgn = 0
            data['end'] = np.nan # not strictly necessary
        if new_payment == True: # add lump sum investment at beginning of year
            end = (bgn + yearly_investment) * (1 + mkt)
        else: # add monthly market growth without lump sum investment
            end = bgn * (1 + mkt)
        data.loc[date, 'end'] = end
        bgn = end
    final_value = round(data['end'].iloc[-1], 2) # calculates the final portfolio value
    return data['end'], final_value

In [ ]:
result, end_val = ls_per_year(df_full)
plt.ylabel('Value ($)')
plt.title('LS Investment Strategy:\n\$20,000 Yearly Using [1927-2021]', fontsize=11, fontweight='bold')
plt.figtext(1.03, 0.5, f'Final Value = ${end_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

In [ ]:
def ls_method(startyear, endyear, amountinvested):
    return ff.loc[startyear:endyear, 'Mkt'].add(1).cumprod().mul(amountinvested)

In [ ]:
def LS_strategy(start, end):
    account_value=0
    ls_final_account=[]
    for i in range(int(start), int(end)+1):
        account_value= account_value+20000
        account_value= ls_method(str(i)+'-01', str(i)+'-12', account_value)[-1]
        ls_final_account.append(account_value)
        return ls_final_account

In [ ]:
LS_graph = np.array(LS_strategy('1927','2021'))
LS_graph.plt.plot(label = 'LS_graph')
plt.xlabel("Date(Year)")
plt.ylabel('Value ($)')
plt.title('LS From 1927 through 2021')
plt.legend()
plt.show()

### DCA Investment Strategy

In [ ]:
# input: fama-french market data with properly spliced timeframe, starting investment
# output: pandas series of the overall value of the portfolio, the final value of the portfolio
monthly_payment = 20000 / 12
def dca_20000_divide_12(data, starting_investment):
    investment_remaining = starting_investment # the amount that is being invested into the DCA strategy
    for date, (mkt, rf) in data[['Mkt', 'RF']].iterrows(): # loop down the data frame of market returns
        if date == data.index[0]:
            dca_bgn = 0
            data['dca_end'] = np.nan # not strictly necessary
        if investment_remaining >= monthly_payment:
            dca_end = (dca_bgn + monthly_payment) * (1 + mkt)
            investment_remaining = investment_remaining - monthly_payment + (investment_remaining * rf) # calculates new cash amount left
        elif investment_remaining < monthly_payment and investment_remaining > 0: # last period of moving cash from savings to market
            dca_end = (dca_bgn + investment_remaining) * (1 + mkt)
            investment_remaining = 0
        else: # all remaining growth through the market after all investments have been made
            dca_end = dca_bgn * (1 + mkt)
        data.loc[date, 'dca_end'] = dca_end
        dca_bgn = dca_end
    final_value = round(data['dca_end'].iloc[-1], 2) # calculates the final portfolio value
    return data['dca_end'], final_value

In [ ]:
q3_result, q3_end_val = dca_20000_divide_12(df_starting_27, 20000)

In [ ]:
q3_result.plot()
plt.ylabel('Value ($)')
plt.title('DCA Investment Strategy:\n\$1666.66666667 Monthly Using \$20,000 Total [1927-2021]', fontsize=11, fontweight='bold')
plt.figtext(0.4, 0.7, 'DCA Investment strategy factors in risk free\n interest rate earned from cash in savings account', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.figtext(1.03, 0.5, f'Final Value = ${q3_end_val}', ha="center", fontsize=8, bbox={"facecolor":"orange", "alpha":0.4, "pad":5})
plt.show()

# Consider all available 20-year holding periods (i.e., 1927 to 1946, 1928 to 1947, and so on)

Which strategy typically has higher final values, how much higher, and why? Discuss.

# Which investing strategy is better overall, LS or DCA?

Discuss, including any limitations of your conclusion.